## Libraires

In [ ]:
import sympy as sp
import numpy as np

## Main Function

In [ ]:
def lagrange_opt_mixed(of, c_eq, c_ineq,maximize=False):
    f = sp.sympify(of)

    # Define the list of equality constraints
    constraints_eq = [sp.sympify(constraint) for constraint in c_eq]

    # Define the list of inequality constraints
    constraints_ineq = [sp.sympify(constraint) for constraint in c_ineq]

    # Define Lagrange multipliers for inequality constraints
    lagrange_multipliers_ineq = [sp.symbols(f'lambda_ineq_{i}') for i in range(len(constraints_ineq))]

    # Define Lagrange multipliers for equality constraints
    lagrange_multipliers_eq = [sp.symbols(f'lambda_eq_{i}') for i in range(len(constraints_eq))]

    # Define the Lagrangian function
    lagrangian = f + sum([lam * constraint for lam, constraint in zip(lagrange_multipliers_ineq, constraints_ineq)]) + \
                     sum([lam * constraint for lam, constraint in zip(lagrange_multipliers_eq, constraints_eq)])

    variables = [sp.symbols('x'), sp.symbols('y')]

    # Compute gradients w.r.t. the variables
    gradients = [sp.diff(lagrangian, var) for var in variables]

    eqs_ineq = gradients + [lam * constraint for lam, constraint in zip(lagrange_multipliers_ineq, constraints_ineq)]

    # Remove Lagrange multipliers from equality constraints
    eqs_eq = [eq.subs({lam: 0 for lam in lagrange_multipliers_eq}) for eq in constraints_eq]
    eqs = eqs_ineq + eqs_eq
    varss = variables + lagrange_multipliers_ineq + lagrange_multipliers_eq
    
    # Solve the system of equations
    solution = sp.solve(eqs, varss)
    
    # Remove the points that don't satisfy the lambda conditions for inequality constraints
    points = []
    for sol in solution:
        if all(np.asarray(sol[len(variables):len(variables) + len(constraints_ineq)]) <= 0):
            points.append((sol[0],sol[1]))      
        else: 
            for sol in solution:
                if all(np.asarray(sol[len(variables):len(variables) + len(constraints_ineq)]) >= 0):
                    points.append((sol[0],sol[1]))

    # Remove the points that don't satisfy the equality constraints
    final_points = []
    for point in points:
        satisfies_eq_constraints = all(abs(con.subs({sp.symbols('x'): point[0], sp.symbols('y'): point[1]})) < 1e-9 for con in constraints_eq)
        if satisfies_eq_constraints:
            final_points.append(point)
            
    subs = [f.subs({sp.symbols('x'): point[0], sp.symbols('y'): point[1]}) for point in final_points]
    
    if maximize:
        optimal = final_points[np.argmax(subs)]
    else:
        optimal = final_points[np.argmin(subs)]
    
    return optimal

## Test Cases

In [2]:
# Test the function
objective_function = 'x**2 - y**2'
equality_constraints = ['x + 2*y + 1']
inequality_constraints = ['x - y -3']
solution = lagrange_opt_mixed(objective_function, equality_constraints, inequality_constraints)
print("Optimal solution:", solution)

Optimal solution: (1/3, -2/3)


In [3]:
# Test the function
objective_function = '(x-1)**2 + y - 2'
equality_constraints = ['y - x - 1']
inequality_constraints = ['x + y - 2']
solution = lagrange_opt_mixed(objective_function, equality_constraints, inequality_constraints)
print("Optimal solution:", solution)

Optimal solution: (1/2, 3/2)
